In [1]:
# First load our CUDA header
from CUDAmanager import Manager
import torch
import time

In [2]:
# Define the test range

# How many run to test
number_of_trials = 25

# Number of keyframes
number_of_poses = [3,  6,  9,  12]

# Number of landmarks per keyframe
number_of_feats = [24, 48, 72, 96]

In [3]:
# CUDA Times will be stored here
cuda_times = {} 
for n in number_of_poses:
    cuda_times.update({n:{}})
    for m in number_of_feats:
        cuda_times[n].update({m:{}})
        manager = Manager(n=n, m=m)
        cuda_times[n][m].update({"meas_size":manager.meas_size})
        cuda_times[n][m].update({"elapsed_times":[]})
        for _ in range(number_of_trials):

            # Initialize the manager
            manager = Manager(n=n, m=m)
            manager.solver.setStepSize(0.1)

            # Get the mean of 10 iterations
            torch.cuda.synchronize()
            t_start = time.monotonic_ns()
            
            manager.solver.step(10)

            torch.cuda.synchronize()
            t_stop = time.monotonic_ns()

            elapsed_time = (t_stop - t_start) / 10.0
            
            # Save the elapsed time to our dictionary
            cuda_times[n][m]["elapsed_times"].append(elapsed_time)

In [4]:
# # CPU Times will be stored here
# cpu_times = {} 
# for n in number_of_poses:
#     cpu_times.update({n:{}})
#     for m in number_of_feats:
#         cpu_times[n].update({m:{}})
#         manager = Manager(n=n, m=m)
#         cpu_times[n][m].update({"meas_size":manager.meas_size})
#         cpu_times[n][m].update({"elapsed_times":[]})
#         print(f"Current Status : {m} - {n}")
#         for _ in range(number_of_trials):
            

#             # Initialize the manager
#             manager = Manager(n=n, m=m)
#             manager.solver.setStepSize(0.1)

#             # Get the mean of 10 iterations
#             t_start = time.monotonic_ns()
#             manager.PyOptimizer.step(observations=manager.simulator.observations)
#             t_stop = time.monotonic_ns()

#             elapsed_time = (t_stop - t_start)
            
#             # Save the elapsed time to our dictionary
#             cpu_times[n][m]["elapsed_times"].append(elapsed_time)

In [5]:
# Execute GTSAM Tests
from StereoGTSAM import GTSAMmanager
# GTSAM Times will be stored here
gtsam_times = {}
for n in number_of_poses:
    gtsam_times.update({n:{}})
    for m in number_of_feats:
        gtsam_manager = GTSAMmanager(n=n, m=n)

        gtsam_times[n].update({m:{}})
        gtsam_times[n][m].update({"meas_size":gtsam_manager.meas_size})
        gtsam_times[n][m].update({"elapsed_times":[]})

        for _ in range(number_of_trials):
            gtsam_manager = GTSAMmanager(n=n, m=n)

            t_start = time.monotonic_ns()
            gtsam_manager.optimizer.optimize()
            t_stop = time.monotonic_ns()

            elapsed_time = (t_stop-t_start)

            print(f"{n} {m} {elapsed_time} {gtsam_manager.optimizer.iterations()}")

            gtsam_times[n][m]["elapsed_times"].append(elapsed_time)

3 24 423592 1
3 24 255358 1
3 24 225600 1
3 24 236057 1
3 24 233174 1
3 24 226408 1
3 24 212685 1
3 24 225723 1
3 24 252635 1
3 24 221622 1
3 24 212498 1
3 24 233895 1
3 24 209713 1
3 24 200744 1
3 24 249416 1
3 24 242508 1
3 24 261171 1
3 24 241864 1
3 24 241932 1
3 24 249542 1
3 24 267102 1
3 24 256849 1
3 24 250608 1
3 24 227121 1
3 24 234802 1
3 48 216594 1
3 48 233008 1
3 48 196361 1
3 48 212125 1
3 48 243212 1
3 48 237822 1
3 48 270339 1
3 48 238691 1
3 48 221681 1
3 48 258023 1
3 48 238395 1
3 48 233711 1
3 48 238499 1
3 48 245089 1
3 48 240528 1
3 48 257981 1
3 48 286257 1
3 48 244987 0
3 48 246246 1
3 48 246378 1
3 48 242913 1
3 48 216918 1
3 48 242131 1
3 48 222140 1
3 48 233058 1
3 72 239498 1
3 72 251152 1
3 72 242820 1
3 72 217781 1
3 72 243154 1
3 72 199069 1
3 72 208843 1
3 72 228385 1
3 72 207361 1
3 72 244436 1
3 72 262095 1
3 72 246022 1
3 72 326969 0
3 72 239469 1
3 72 245555 1
3 72 270318 1
3 72 238341 1
3 72 254938 1
3 72 217223 1
3 72 215136 1
3 72 235327 1
3 72 2

In [ ]:
import numpy as np
# Finally create the overleaf table
with open("table.tex", "w") as tbl:
    # Begin table environment
    tbl.write("\\begin{table*}[t]\n")
    tbl.write("\\small\n")
    tbl.write("\\centering\n")
    tbl.write("\\caption{Timing Benchmark}\n")
    
    # Dynamically create the column format string
    col_format = "|l|" + ("c" * len(number_of_feats) + "|") * len(number_of_poses)

    tbl.write("\\resizebox{\\linewidth}{!}{\n")
    tbl.write(f"\\begin{{tabular}}{{{col_format}}}\n")
    tbl.write("\\toprule\n")
    tbl.write("\\midrule\n")
    
    # Write the header row
    header = ["Number of Keyframes "]
    for idx, key_frame in enumerate(number_of_poses):
        header = header + ["& \\multicolumn{"  + str(len(number_of_feats)) + "}{c|}{$" + str(key_frame) + "$}"]

    tbl.write("".join(header) + " \\\\\n")
    tbl.write("\\midrule\n")

    subheader = ["Feature per KF "]
    for _ in number_of_poses:
        for num_of_feat in number_of_feats:
            subheader = subheader + ["& $" + str(num_of_feat) + "$ "]
    tbl.write("".join(subheader) + " \\\\\n")
    tbl.write("\\midrule\n")

    line = ["Measurement Size"]
    for n in number_of_poses:
        for m in number_of_feats:
            line = line + ["& " + str(cuda_times[n][m]["meas_size"]) + " "]
    tbl.write("".join(line) + " \\\\\n")
    tbl.write("\\midrule\n")

    line = ["Elapsed Time GTSAM (ms):"]
    for n in number_of_poses:
        for m in number_of_feats:
            elapsed_time = np.median(np.array(gtsam_times[n][m]["elapsed_times"])) * 1e-6
            elapsed_time = f"{elapsed_time:.2f}"
            line = line + ["& " + elapsed_time + " "]
    tbl.write("".join(line) + " \\\\\n")
    tbl.write("\\midrule\n")

    line = ["Elapsed Time GPU (ms):"]
    for n in number_of_poses:
        for m in number_of_feats:
            elapsed_time = np.median(np.array(cuda_times[n][m]["elapsed_times"])) * 1e-6
            elapsed_time = f"{elapsed_time:.2f}"
            line = line + ["& " + elapsed_time + " "]
    tbl.write("".join(line) + " \\\\\n")
    tbl.write("\\midrule\n")


    # line = ["Elapsed Time CPU:"]
    # for n in number_of_poses:
    #     for m in number_of_feats:
    #         elapsed_time = np.median(np.array(gtsam_times[n][m]["elapsed_times"])) * 1e-6
    #         elapsed_time = f"{elapsed_time:.2f}"
    #         line = line + ["& " + elapsed_time + " "]
    # tbl.write("".join(line) + " \\\\\n")
    # tbl.write("\\midrule\n")
    
    # # Fill rows with RMSE values
    # for iter in self.DVOResults.keys():
    #     for arch in self.DVOResults[iter].keys():
    #         row_data = [f"{arch.replace('_', '$\_$')} {iter.replace('_', '$\_$')}"]
    #         for seq in self.sequences:
    #             err_arr = self.DVOResults[iter][arch][seq]
    #             # Join all RMSE values for the sequence into a single string
    #             rmse_str = ", ".join([f"{rmse:.3f}" for rmse in err_arr])
    #             row_data.append(rmse_str)
    #         tbl.write(" & ".join(row_data) + " \\\\\n")
    #     tbl.write("\\midrule\n")
    
    # Finish table
    tbl.write("\\bottomrule\n")
    tbl.write("\\end{tabular}}\n")
    tbl.write("\\end{table*}\n")